In [2]:
import numpy as np
import heapq

class Server:
    def __init__(self, server_id, service_rate):
        self.server_id = server_id
        self.service_rate = service_rate
        self.current_time = 0  # Tracks the server's next available time

    def process_job(self, arrival_time):
        """
        Processes a job and returns the completion time.
        """
        start_time = max(self.current_time, arrival_time)
        service_time = np.random.exponential(1 / self.service_rate)
        completion_time = start_time + service_time
        self.current_time = completion_time
        return completion_time


class Router:
    def __init__(self, servers):
        self.servers = servers

    def route(self, arrival_time):
        """
        Selects a server for the incoming job.
        Default routing strategy: chooses the server with the earliest availability.
        """
        return min(self.servers, key=lambda s: s.current_time)


class KServerQueueingSystem:
    def __init__(self, num_servers, service_rates, arrival_rate):
        # Ensure servers are sorted by service rates in descending order
        sorted_service_rates = sorted(service_rates, reverse=True)
        self.num_servers = num_servers
        self.servers = [Server(i, sorted_service_rates[i]) for i in range(num_servers)]
        self.router = Router(self.servers)  # Instantiate a Router
        self.arrival_rate = arrival_rate
        self.event_queue = []  # Priority queue for events (arrival and completion)
        self.time = 0

    def simulate(self, num_jobs):
        """
        Simulates the queuing system for a given number of jobs.
        """
        # Generate inter-arrival times and compute arrival times
        inter_arrival_times = np.random.exponential(1 / self.arrival_rate, num_jobs)
        arrival_times = np.cumsum(inter_arrival_times)

        # Add all arrival events to the event queue
        for arrival_time in arrival_times:
            heapq.heappush(self.event_queue, (arrival_time, 'arrival'))

        completed_jobs = 0
        job_completion_times = []

        while completed_jobs < num_jobs:
            event_time, event_type = heapq.heappop(self.event_queue)
            self.time = event_time

            if event_type == 'arrival':
                # Route the job to a server
                selected_server = self.router.route(self.time)
                completion_time = selected_server.process_job(self.time)

                # Add the completion event to the event queue
                heapq.heappush(self.event_queue, (completion_time, 'completion'))
            elif event_type == 'completion':
                # Job is completed
                completed_jobs += 1
                job_completion_times.append(self.time)

        return job_completion_times


# Example usage:
if __name__ == "__main__":
    # Define parameters
    k = 3  # Number of servers
    service_rates = [1.5, 2.0, 1.0]  # Service rates (µi) for each server
    arrival_rate = 3.0  # Job arrival rate (λ)
    num_jobs = 1000  # Number of jobs to simulate

    # Initialize and run the simulation
    queue_system = KServerQueueingSystem(k, service_rates, arrival_rate)
    completion_times = queue_system.simulate(num_jobs)

    # Print results
    print(f"Simulated {num_jobs} jobs with {k} servers.")
    print(f"Mean job completion time: {np.mean(completion_times):.2f}")
    print(f"Maximum job completion time: {np.max(completion_times):.2f}")


Simulated 1000 jobs with 3 servers.
Mean job completion time: 169.71
Maximum job completion time: 329.81
